In [2]:
import pandas as pd
import numpy as np
import requests

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/mikha122/mypojects/main/Football%20Project/all_merged.csv',index_col = 0)

In [4]:
data

,gameID,homeTeamID,awayTeamID,homeProbability,drawProbability,awayProbability,homeGoalsHalfTime,awayGoalsHalfTime,teamID_home,goals_home,...,goals_away,xGoals_away,shots_away,shotsOnTarget_away,deep_away,ppda_away,fouls_away,corners_away,yellowCards_away,redCards_away
0,81,89,82,0.2843,0.3999,0.3158,1,0,89,1,...,0,0.674600,9,4,10,8.2188,12,2,3,0
1,82,73,71,0.3574,0.3500,0.2926,0,0,73,0,...,1,0.782253,7,3,2,11.8462,13,3,4,0
2,83,72,90,0.2988,0.4337,0.2675,0,1,72,2,...,2,0.557892,11,5,4,17.1579,13,2,2,0
3,84,75,77,0.6422,0.2057,0.1521,3,0,75,4,...,2,1.459460,11,5,6,9.5556,17,3,4,0
4,85,79,78,0.1461,0.2159,0.6380,0,1,79,1,...,3,2.109750,11,7,10,10.6250,20,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,14810,87,78,0.7516,0.1730,0.0754,1,0,87,2,...,0,0.772317,5,4,1,16.7500,8,1,2,0
2656,14811,88,72,0.7906,0.1483,0.0611,2,0,88,5,...,0,1.072580,8,3,3,19.1154,10,5,2,0
2657,14812,238,92,0.2153,0.4670,0.3177,1,0,238,1,...,0,0.558508,10,3,10,9.3529,1,9,1,0
2658,14813,81,74,0.5303,0.2337,0.2360,2,0,81,3,...,0,1.351790,17,5,10,6.1667,9,3,3,0


In [5]:
# here, i want to implement 3 different machine learning algorithms in order to try to predict football results. Each model will be responsible for a different part of the 
# these models will try to predict away game goals, home team goals and the winner of the game. 
# albeit realistically, we don't have to predict the winner of the game, instead just take a few if and else statements which will then compare the results and say which side won.

In [6]:
data_copy = data.copy()

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
#For this task i will use a SGD Classifier for both sides.
from sklearn.linear_model import SGDClassifier

r = SGDClassifier()

goals_away = data['goals_away']
goals_home = data['goals_home']
result_home = data['result_home']
data = data.drop(columns=['gameID','result_home','goals_away','goals_home','teamID_home','teamID_away','ppda_home','ppda_away'])


In [9]:
X_train, X_test, y_train, y_test = train_test_split(data, goals_home, random_state=42)
X_result_train, X_result_test, y_result_train, y_result_test = train_test_split(data,result_home,random_state=42)

In [10]:
# this cell will predict home goals
# hence, target is goals_home

model = r.fit(X_train,y_train)

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [12]:
X_outcome = model.predict(X_test)

In [13]:
X_outcome = X_outcome.reshape(-1,1)


In [14]:
accuracy = accuracy_score(y_test, X_outcome)
accuracy

0.3082706766917293

Clearly, the algorithm isn't perfect, having achieved 30.9% accuracy. Pretty lame

In [15]:
precision = precision_score(y_test,X_outcome,average='micro')
precision

0.3082706766917293

In [16]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'penalty': ['l2','l1'],
    'alpha': [0.0001,0.001,0.01,0.0005,0.005,0.05],
    'learning_rate': ['constant','optimal','adaptive']
}

GS = GridSearchCV(r, param_grid=param_grid, cv=10)

In [17]:
GS.fit(X_train, y_train)

/Users/mikhail/miniforge3/envs/tensorflow/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
/Users/mikhail/miniforge3/envs/tensorflow/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/mikhail/miniforge3/envs/tensorflow/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/mikhail/miniforge3/envs/tensorflow/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warning

GridSearchCV(cv=10, estimator=SGDClassifier(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.0005, 0.005, 0.05],
                         'learning_rate': ['constant', 'optimal', 'adaptive'],
                         'penalty': ['l2', 'l1']})

In [18]:
GS.best_estimator_

SGDClassifier(alpha=0.0005, penalty='l1')

So, alpha of 0.001 and l1 penalty are the best parameters for the job.

In [19]:
grid_search_outcome = GS.predict(X_test)

In [20]:
new_accuracy = accuracy_score(y_test,grid_search_outcome)
new_accuracy

0.45413533834586467

so, GridSearch was able to add about 11% onto the accuracy of the model. That is good, but the model still isn't great

In [21]:
# I think it's best to leave some other optimisations till the end, after we see how the models compare.

In [22]:
away_r = SGDClassifier()
X_train_away, X_test_away, y_train_away, y_test_away = train_test_split(data, goals_away, random_state=42)

In [23]:
away_model = away_r.fit(X_train_away, y_train_away)

In [24]:
X_outcome_away = away_model.predict(X_test_away)

In [25]:
away_accuracy = accuracy_score(y_test_away,X_outcome_away)
away_accuracy

0.5097744360902255

In [26]:
away_gridsearch = GridSearchCV(away_r,param_grid,cv=10)
away_gridsearch.fit(X_train_away, y_train_away)

/Users/mikhail/miniforge3/envs/tensorflow/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
/Users/mikhail/miniforge3/envs/tensorflow/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/mikhail/miniforge3/envs/tensorflow/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/mikhail/miniforge3/envs/tensorflow/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warning

GridSearchCV(cv=10, estimator=SGDClassifier(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.0005, 0.005, 0.05],
                         'learning_rate': ['constant', 'optimal', 'adaptive'],
                         'penalty': ['l2', 'l1']})

In [27]:
away_gridsearch.best_estimator_

SGDClassifier(alpha=0.001, penalty='l1')

In [28]:
away_GS_outcome = away_gridsearch.predict(X_test_away)
GS_away_accuracy = accuracy_score(y_test_away,away_GS_outcome)

In [29]:
GS_away_accuracy

0.49473684210526314

So, the tuned away model can predict more accurately. That is interesting since the data is very similar. It could mean that this is just the case for this small sample but it is difficult to say.

In [30]:
# now i will try to use the away_GS_outcome and home grid_search_outcome in order to see the accuracy of the games
predicted_results = []
for i in range(len(grid_search_outcome)):
    if grid_search_outcome[i] > away_GS_outcome[i]:
        predicted_results.append(2)
    elif grid_search_outcome[i] == away_GS_outcome[i]:
        predicted_results.append(1)
    elif grid_search_outcome[i] < away_GS_outcome[i]:
        predicted_results.append(0)

In [31]:
# now to find out how accurate the overall game prediction is
game_accuracy = accuracy_score(y_result_test, predicted_results)
game_accuracy

0.6

The overall accuracy is therefore 60%. This isn't great, however, due to the nature of the events, there are 3 possibilities, draw, win or loss. Therefore, the algorithm is actually a better prediction than guessing. 

In [32]:
Tot_Brighton = pd.read_csv('/Users/mikhail/Jupyter/Jupyter-1/Projects/Football Project/Leeds_Newcastle.csv',index_col=0)

In [33]:
Tot_Brighton

,homeTeamID,awayTeamID,homeProbability,drawProbability,awayProbability,homeGoalsHalfTime,awayGoalsHalfTime,xGoals_home,shots_home,shotsOnTarget_home,...,yellowCards_home,redCards_home,xGoals_away,shots_away,shotsOnTarget_away,deep_away,fouls_away,corners_away,yellowCards_away,redCards_away
0,89,82,0.363514,0.281386,0.3551,1.0,0.714286,1.057323,12.142857,3.714286,...,1.571429,0.142857,1.191511,11.142857,4.428571,5.0,12.571429,3.428571,1.857143,0.0


In [34]:
Tot_result = GS.predict(Tot_Brighton)
Brighton_result = away_gridsearch.predict(Tot_Brighton)

In [35]:
print(Tot_result,Brighton_result)

[2] [1]
